In [3]:
import openai
import os
import random
import time

openai.api_key = os.environ["openaipersonal"]

In [4]:
sudoku_str = """_21___4_6
_8792____
__486__27
7____985_
5_92____1
2__7_36__
_7__8236_
892_3__7_
_3___7__8""".split("\n")

row_indices = "abcdefghi"
col_indices = "БВГДЕЖЗИК"
square_indices = "αβγδεζηθι"

new_sudoku_str = []
for row in range(9):
    new_row = ""
    for col in range(9): 
        new_row += f"{row_indices[row]}{col_indices[col]}{square_indices[(row//3)*3 + col//3]}: {sudoku_str[row][col]} | "
    new_row = "".join(new_row)
    new_sudoku_str.append(new_row)
    if row in [2, 5]:
        new_sudoku_str.append("====")
    else:
        new_sudoku_str.append("----")

grid = "\n".join(new_sudoku_str)

grid += "\n Recently tested groups: a, b, c, Б, В, Г, α, β, γ, δ."
print(len(grid))
print(grid)

837
aБα: _ | aВα: 2 | aГα: 1 | aДβ: _ | aЕβ: _ | aЖβ: _ | aЗγ: 4 | aИγ: _ | aКγ: 6 | 
----
bБα: _ | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | 
----
cБα: _ | cВα: _ | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: _ | cЗγ: _ | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: _ | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: _ | eГδ: 9 | eДε: 2 | eЕε: _ | eЖε: _ | eЗζ: _ | eИζ: _ | eКζ: 1 | 
----
fБδ: 2 | fВδ: _ | fГδ: _ | fДε: 7 | fЕε: _ | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: _ | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: _ | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι: _ | iКι: 8 | 
----
 Recently tested groups: a, b, c, Б, В, Г, α, β, γ, δ.


In [5]:
import re

def find_latest_grid(text):
    pattern = (r"(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}"
               r"\n={4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}"
               r"\n={4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}")

    matches = list(re.finditer(pattern, text))
    
    if matches:
        return matches[-1].group(0)  # Returns the latest (last) match
    return None

In [32]:
row_indices = "abcdefghi"
col_indices = "БВГДЕЖЗИК"
square_indices = "αβγδεζηθι"

def process_str_grid(sudoku_grid):
    sudoku_grid = find_latest_grid(sudoku_grid)
    rows = [row for row in sudoku_grid.strip().split('\n') if not row.startswith('-') and not row.startswith('=')]
    processed_grid = [[int(cell.split(': ')[1]) if cell.split(': ')[1] != '_' else 0 for cell in row.rstrip(' |').split(' | ')] for row in rows]
    return processed_grid


def is_valid_sudoku(grid):
    def is_valid(num, row, col):
        for i in range(9):
            if grid[row][i] == num: return False
            if grid[i][col] == num: return False
            if grid[3*(row//3) + i//3][3*(col//3) + i%3] == num: return False
        return True

    def helper():
        for row in range(9):
            for col in range(9):
                if grid[row][col] == 0:
                    for num in range(1, 10):
                        if is_valid(num, row, col):
                            grid[row][col] = num
                            if helper(): return True
                            grid[row][col] = 0
                    return False
        return True
    
    # Check rows
    for row in range(9):
        numbers_seen = set()
        for col in range(9):
            num = grid[row][col]
            if num != 0:
                if num in numbers_seen:
                    return False, f'Contradiction at row {row_indices[row]}'
                numbers_seen.add(num)

    # Check columns
    for col in range(9):
        numbers_seen = set()
        for row in range(9):
            num = grid[row][col]
            if num != 0:
                if num in numbers_seen:
                    return False, f'Contradiction at column {col_indices[col]}'
                numbers_seen.add(num)

    # Check 3x3 squares
    for square in range(9):
        numbers_seen = set()
        start_row = (square // 3) * 3
        start_col = (square % 3) * 3
        for row in range(start_row, start_row + 3):
            for col in range(start_col, start_col + 3):
                num = grid[row][col]
                if num != 0:
                    if num in numbers_seen:
                        return False, f'Contradiction at square {square_indices[square]}'
                    numbers_seen.add(num)
    
    # if everything looks good, check if the sudoku is solvable
    if not helper():
        return False, ""
    return True, "No contradiction"

In [43]:
system_prompt = """You are a very sophisticated LLM. 
Your current task is to train a baby LLM in precise, acurate, logical thinking.
For this exercise, you are demonstrating the logical steps to solve a sudoku.
It is important that you always write out the logical steps in full, because the baby LLM you are training needs to understand them correctly.
It is very important that you don't skip steps. 
It is absolutely crucial that you first list the predicates, and only then the conclusions. 
I.e. always say, "Because of X and Y, therefore Z" and never say "Z, because X and Y".
"""

def generate_user_prompt(grid):
#     groups = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'К', 'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι']
#     sampled_elements = random.sample(groups, 8)
#     print("generating prompt with groups ", sampled_elements)
#     str_groups = ", ".join(sampled_elements)
    
    user_prompt = f"""Here is the sudoku that needs to be solved:

=====
=====
=====
{grid}


It is written in a rather peculiar way, to aid the understanding of the baby AI you're teaching.

There are added indices to each square, indicating the row, the column, and the 3x3 square each square belongs to.
Rows have an index from a, b, c, d, e, f, g, h, i, 
columns have an index from Б, В, Г, Д, Е, Ж, З, И, К, 
3x3 squares have an index from α, β, γ, δ, ε, ζ, η, θ, ι. 
Empty squares are represented as "_".
We call each column, row, and 3x3 square a "group". 
Each group should have the numbers from 1 to 9 exactly. 
Every group has exactly 9 squares.  
A group CANNOT contain the same number twice. 
A group MUST contain every number from 1 to 9.

Using this index, you can tell exactly to which groups a square belongs.
Example: gБη belongs to groups g, Б, η. 
Example: dЕε belongs to groups d, Е, and ε. 

Additionally: groups intersect. The squares which have the indices of two groups, belong to both groups.
Example: group a intersects group α in the squares aБα, aВα, aГα.
Example: group g intersects group Ж in square gЖθ.
Example: group ι intersects group К in squares gКι, hКι, iКι.
Example: column Е intersects row f only in the square fЕε.

For reference, here are all the groups and all the squares they contain.
Group a:  aБα, aВα, aГα, aДβ, aЕβ, aЖβ, aЗγ, aИγ, aКγ
Group b:  bБα, bВα, bГα, bДβ, bЕβ, bЖβ, bЗγ, bИγ, bКγ
Group c:  cБα, cВα, cГα, cДβ, cЕβ, cЖβ, cЗγ, cИγ, cКγ
Group d:  dБδ, dВδ, dГδ, dДε, dЕε, dЖε, dЗζ, dИζ, dКζ
Group e:  eБδ, eВδ, eГδ, eДε, eЕε, eЖε, eЗζ, eИζ, eКζ
Group f:  fБδ, fВδ, fГδ, fДε, fЕε, fЖε, fЗζ, fИζ, fКζ
Group g:  gБη, gВη, gГη, gДθ, gЕθ, gЖθ, gЗι, gИι, gКι
Group h:  hБη, hВη, hГη, hДθ, hЕθ, hЖθ, hЗι, hИι, hКι
Group i:  iБη, iВη, iГη, iДθ, iЕθ, iЖθ, iЗι, iИι, iКι
Group Б:  aБα, bБα, cБα, dБδ, eБδ, fБδ, gБη, hБη, iБη
Group В:  aВα, bВα, cВα, dВδ, eВδ, fВδ, gВη, hВη, iВη
Group Г:  aГα, bГα, cГα, dГδ, eГδ, fГδ, gГη, hГη, iГη
Group Д:  aДβ, bДβ, cДβ, dДε, eДε, fДε, gДθ, hДθ, iДθ
Group Е:  aЕβ, bЕβ, cЕβ, dЕε, eЕε, fЕε, gЕθ, hЕθ, iЕθ
Group Ж:  aЖβ, bЖβ, cЖβ, dЖε, eЖε, fЖε, gЖθ, hЖθ, iЖθ
Group З:  aЗγ, bЗγ, cЗγ, dЗζ, eЗζ, fЗζ, gЗι, hЗι, iЗι
Group И:  aИγ, bИγ, cИγ, dИζ, eИζ, fИζ, gИι, hИι, iИι
Group К:  aКγ, bКγ, cКγ, dКζ, eКζ, fКζ, gКι, hКι, iКι
Group α:  aБα, aВα, aГα, bБα, bВα, bГα, cБα, cВα, cГα
Group β:  aДβ, aЕβ, aЖβ, bДβ, bЕβ, bЖβ, cДβ, cЕβ, cЖβ
Group γ:  aЗγ, aИγ, aКγ, bЗγ, bИγ, bКγ, cЗγ, cИγ, cКγ
Group δ:  dБδ, dВδ, dГδ, eБδ, eВδ, eГδ, fБδ, fВδ, fГδ
Group ε:  dДε, dЕε, dЖε, eДε, eЕε, eЖε, fДε, fЕε, fЖε
Group ζ:  dЗζ, dИζ, dКζ, eЗζ, eИζ, eКζ, fЗζ, fИζ, fКζ
Group η:  gБη, gВη, gГη, hБη, hВη, hГη, iБη, iВη, iГη
Group θ:  gДθ, gЕθ, gЖθ, hДθ, hЕθ, hЖθ, iДθ, iЕθ, iЖθ
Group ι:  gЗι, gИι, gКι, hЗι, hИι, hКι, iЗι, iИι, iКι

Please solve it with the following steps:

1) Select 6 groups from all groups in the sudoku. 
Alternate between row, column, 3x3 square. 
Choose the groups that contain a low, but non-zero number of empty squares.
You CANNOT choose any groups mentioned in the recently tested groups list.
For each of the 6 groups you chose, name the cells they contain by index.

1.1) For each of the selected groups above:
* name the numbers they contain, with the respective index;
* the numbers remaining
* which cells are empty (by index)
* how many cells are how many are filled in and how many are empty
Your job here is to check the list you created in the previous step against the sudoku grid above.

Example:
"Group θ -  numbers gДθ: 1, gЕθ: _, gЖθ: _, hДθ: 7, hЕθ: _, hЖθ: 4, iДθ: _, iЕθ: 3, iЖθ: 8;
Missing the numbers - 2, 5, 6, 9.
The empty squares in group θ are gЕθ, gЖθ, hЕθ, iДθ.
The filled in numbers are 5, the empty squares are 4."

Example:
"Group δ - contains the numbers dБδ: 2, dВδ: _, dГδ: _, eБδ: 8, eВδ: 9, eГδ: _, fБδ: 5, fВδ: 4, fГδ: 3; 
Missing the numbers - 1, 6, 7. 
The empty squares in group δ are dВδ, dГδ, eГδ.
The filled in numbers are 6, the empty squares are 3."

2) Pick and target one specific group from the ones you've chosen. 
We will call it the "target group". Pick one that has the least number of empty squares, but that has at least 1 empty square.  
Name all of it's empty squares, and the numbers we need to fill them with. 
Name all groups that intersect with its empty squares. 
The intersecting groups should be an index in at least one of the empty squares we mentioned.
Here, it is very important to be very precise when listing the intersecting groups - list them all, but be careful not to list any others.
Remember, if our target group is a row, it cannot intersect with other rows, if it's a column, with other columns, and if it's a square, with other squares.

Example:
"Among our chosen groups, the groups with the smallest non-zero number of empty squares have 2 empty squares.
Those are groups g and ζ. 
Let's select as our target group, g.
It contains the empty squares gГη, gЗι and intersects groups Г, η, З, ι.". 
Here, we find the intersecting groups by looking at the indices of the empty squares. 
gГη has indices g, Г, η, therefore Г and η are intersecting groups.

Example:
"Among our chosen groups, the groups with the smallest non-zero number of empty squares have 4 empty squares.
Those are groups Б, В, f, i. 
Let's select as our target group, Б.
It contains the empty squares aБα, bБα, cБα, iБη and intersects groups a, b, c, i, α, η."


"""
    return user_prompt


user_prompt_part2 = """3) Now we have the missing numbers from our target group, the empty squares and the intersecting groups.
We want to know what numbers we can fill in in our target group. 
We will exclude some possibilities for some squares.

Do the following:
* list the missing numbers and in which squares we need to fill them in, as well as the intersecting groups.
* for each intersecting group:
    ** list all the values it contains in full notation
    ** list all the values it contains in simple notation, then write which of our empty squares it intersects
    ** then write if it excludes any value for the empty squares in our target group.

Examples: 
"We need to fill in numbers 1, 2, 7 in squares dВδ, eБδ, eГδ. The intersecting groups are В, e, d.
Intersecting group В: aВα: 9, bВα: 1, cВα: _, dВδ: _, eВδ: _, fВδ: _, gВη: 8, hВη: _, iВη: 2;
Intersecting group В contains 1, 2, 8, 9.
It intersects square dВδ, therefore square dВδ cannot be 1, 2, 8 or 9."

"Intersecting group d: dБδ: 5, dВδ: _, dГδ: _, dДε: _, dЕε: 6, dЖε: _, dЗζ: 8, dИζ: _, dКζ: _;
Intersecting group d contains 5, 6, 8.
It contains square dВδ, therefore square dВδ cannot be 5, 6, 8."

Make sure to mention every intersecting group from the previous step exactly once, and in order.

3.1) For every empty square in our target group, provide a union of all the excluded numbers listed in step 3.

Example:
"Referring to step 3, square dВδ cannot be 1, 2, 8, 9, 5, 6."

After completing step 3.1, stop and wait for further instructions.

4) Combine all your observations. 
For each empty square in the target group, what are the numbers we need to fill in in our target group?
Those are the "initial possibilities".
Initially, they are the same for every empty square in our target group.
For each empty square in the target group, what are the numbers we've excluded in step 3 from the initial possibilities?
Those are the "excluded possibilities".
Remember, every empty square should be mentioned twice in the intersecting groups in step 3.
Finally, for every square, what are the final possibilities?
Those are calculated by subtracting the two sets : "initial possibilities" - "final possibilities".

Example:
Square X: initial possibilities [1, 5, 9]. From them, for square X, we have excluded the possibilities [1, 5]. 
Therefore, the final possibilities for X are [9].
Square Y: initial possibilities [1, 5, 9]. From them, for square Y, we have excluded the possibilities [1].
Therefore, the final possibilities for Y are [5, 9].
Square Z: initial possibilities [1, 5, 9]. From them, for square Y, we have excluded the possibilities [].

Afterwards, if the final possibilities for any given empty square are only 1 number, we can fill it in!

Example: 
"We can fill in number 9 in square X."
Or: 
"We cannot fill in any number with certainty. Proceeding with steps 5 and 6."

4.1) If one of the numbers from the initial possibilities list is only possible for one of the empty squares, we can also fill it in.
Otherwise, proceed to the next steps.

4.2) If you have successfully filled in a number, we can quickly go back to our results in step 4, 
and exclude this number from the final possibilities for the other squares.
Thus, we might be able to fill in an additional number. 
If there is any other square with only 1 remaining number in the final possibilities, also fill it in.
Going back to the same example:
"There is only one possibility for square X, we can fill it in! Square X = 9.
Therefore, we can remove 9 from the final possibilities for all the other squares:
The final possibilities for square Y are [5]. We can fill it in!
The final possibilities for square Z are [5, 9]. Proceeding with steps 5 and 6."

Important - not always will you be able fill in a number in your target group. 
That is fine, in that case, proceed to steps 5 and 6.
You are demonstrating the correct logical process to the baby AI. We have to be patient with logical problems such as sudoku.

Important - if you have exhausted all possibilities for a given square, DO NOT attempt to fill it in, you've made a mistake somewhere. 
Ignore this square and continue with the steps.

Likewise, if you find that the same number is the only final possibility for two squares, you have made a mistake somewhere. 
Just say - "Sanity check - we shouldn't get such and such result" and continue to step 6.

5) Exactly reproduce the "recently tested groups" you got in your instructions.
Afterwards, write them out again, except begin with the target group we chose in the first place, and drop the last one from the list.
The list is exactly the same, except we add one group in the front and remove one group from the end of the list.

For example, if you were given the list:
"Recently tested groups (old): i, З, Е, ζ, К, Б, α, c, g, β.", and you tested group Ж, you should write:
"Recently tested groups: Ж, i, З, Е, ζ, К, Б, α, c, g."

6) Finally, we will finish this iteration of the steps by filling in our work. 
First repeat what number(s) you want to fill in which squares; if there are none, say so.
Write the sudoku grid with the new number(s) filled in the correct squares.
Use exactly the same format for the sudoku grid, including coordinates and grid.
Otherwise, write the sudoku grid as received from the instructions with no new numbers filled in.
In both cases, there MUST be a entire sudoku grid, otherwise the baby AI will not understand our process.
Do not write a partial grid, write until you have the entire grid.

After the sudoku grid, again repeat the recently tested groups from step 5:
"Recently tested groups: ..." 

Do not say anything else besides the sudoku grid and the recently tested groups in this step."""

In [44]:
answers = []
grids = [grid]
new_grid = grid
gpt_failures = 0

def take_steps(maximum_steps=50, current_step=0):
    global gpt_failures
    if current_step == maximum_steps:
        print("maximum steps reached, ending process :)")
        return
    
    print(f"now on step {current_step}, current time is {time.strftime('%H:%M')}")    
    # time.sleep(20) # because gpt is prissy otherwise

    user_prompt = generate_user_prompt(grids[-1])
    
    try:
        response = openai.ChatCompletion.create(
                      model="gpt-4",
                      messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": user_prompt},
                        ],
                        temperature=0,
                    )

        answer = response['choices'][0]['message']['content']
        print("=======\n", answer, "\n=======\n", )
        answers.append(answer)
        
        # time.sleep(20)
        
        response = openai.ChatCompletion.create(
                      model="gpt-4",
                      messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": user_prompt},
                            {"role": "assistant", "content": answer},
                            {"role": "user", "content": user_prompt_part2}
                        ],
                        temperature=0,                    
        
        
        )
        answer2 = response['choices'][0]['message']['content']
        print("=======\n", answer2, "\n=======\n", )
        answers.append(answer2)
        
        new_grid = answer2[-842:]
        # print("\n\n", new_grid, "\n\n")
        grids.append(new_grid)
        print("remaining empty squares: ", new_grid.count("_"))
        
        try:
            processed_grid = process_str_grid(new_grid)
            correct, problem = is_valid_sudoku(processed_grid)

        except:
            print("could not find/process grid")
            return
            
        if correct:
            print("it's a valid grid")
        else:
            print(f"invalid grid, {problem}")
            return
            
        if not new_grid.count("_"):
            print("🎉🎊🥳🚀✨ Complete! ✨🚀🥳🎊🎉")
            return
        
        take_steps(maximum_steps, current_step+1)
    
    except Exception as e:
        print(f"following error encountered calling gpt...", e, "\n")
        gpt_failures += 1
        if gpt_failures > 0.6 * current_step and gpt_failures > 2:
            return

        # time.sleep(180)  # no, stop, you're having too much fun, reee
        take_steps(maximum_steps, current_step)
        
take_steps(maximum_steps = 50)

now on step 0, current time is 02:20
 1) Let's select six groups from the sudoku that contain a low, but non-zero number of empty squares. We will alternate between row, column, and 3x3 square. We will not choose any groups mentioned in the recently tested groups list. 

The selected groups are: 

- Group d: dБδ, dВδ, dГδ, dДε, dЕε, dЖε, dЗζ, dИζ, dКζ
- Group З: aЗγ, bЗγ, cЗγ, dЗζ, eЗζ, fЗζ, gЗι, hЗι, iЗι
- Group e: eБδ, eВδ, eГδ, eДε, eЕε, eЖε, eЗζ, eИζ, eКζ
- Group И: aИγ, bИγ, cИγ, dИζ, eИζ, fИζ, gИι, hИι, iИι
- Group f: fБδ, fВδ, fГδ, fДε, fЕε, fЖε, fЗζ, fИζ, fКζ
- Group ζ: dЗζ, dИζ, dКζ, eЗζ, eИζ, eКζ, fЗζ, fИζ, fКζ

1.1) For each of the selected groups above, let's name the numbers they contain, the numbers remaining, which cells are empty, and how many cells are filled in and how many are empty.

- Group d: numbers dБδ: 7, dВδ: _, dГδ: _, dДε: _, dЕε: _, dЖε: 9, dЗζ: 8, dИζ: 5, dКζ: _; Missing the numbers - 1, 2, 3, 4, 6. The empty squares in group d are dВδ, dГδ, dДε, dЕε, dКζ.

 3) Now we have the missing numbers from our target group, the empty squares and the intersecting groups.

We need to fill in numbers 1, 4, 5, 9 in squares gБη, gГη, gДθ, gКι. The intersecting groups are Б, Г, Д, К, η, θ, ι.

Intersecting group Б: aБα: _, bБα: _, cБα: _, dБδ: 7, eБδ: 5, fБδ: 2, gБη: _, hБη: 8, iБη: _;
Intersecting group Б contains 2, 5, 7, 8.
It intersects square gБη, therefore square gБη cannot be 2, 5, 7, 8.

Intersecting group Г: aГα: 1, bГα: 7, cГα: 4, dГδ: _, eГδ: 9, fГδ: _, gГη: _, hГη: 2, iГη: _;
Intersecting group Г contains 1, 2, 4, 7, 9.
It intersects square gГη, therefore square gГη cannot be 1, 2, 4, 7, 9.

Intersecting group Д: aДβ: _, bДβ: 9, cДβ: 8, dДε: _, eДε: 2, fДε: 7, gДθ: _, hДθ: _, iДθ: _;
Intersecting group Д contains 2, 7, 8, 9.
It intersects square gДθ, therefore square gДθ cannot be 2, 7, 8, 9.

Intersecting group К: aКγ: 6, bКγ: _, cКγ: 7, dКζ: _, eКζ: 1, fКζ: _, gКι: _, hКι: _, iКι: 8;
Intersecting group К contains 1, 6, 7, 8.
It intersects 

 1) Let's select 6 groups from all groups in the sudoku. We will alternate between row, column, 3x3 square and choose the groups that contain a low, but non-zero number of empty squares. We cannot choose any groups mentioned in the recently tested groups list. 

The selected groups are: 

Group e: eБδ, eВδ, eГδ, eДε, eЕε, eЖε, eЗζ, eИζ, eКζ
Group f: fБδ, fВδ, fГδ, fДε, fЕε, fЖε, fЗζ, fИζ, fКζ
Group i: iБη, iВη, iГη, iДθ, iЕθ, iЖθ, iЗι, iИι, iКι
Group Д: aДβ, bДβ, cДβ, dДε, eДε, fДε, gДθ, hДθ, iДθ
Group Е: aЕβ, bЕβ, cЕβ, dЕε, eЕε, fЕε, gЕθ, hЕθ, iЕθ
Group ζ: dЗζ, dИζ, dКζ, eЗζ, eИζ, eКζ, fЗζ, fИζ, fКζ

1.1) For each of the selected groups above, we will name the numbers they contain, with the respective index; the numbers remaining; which cells are empty (by index); and how many cells are filled in and how many are empty.

Group e - contains the numbers eБδ: 5, eВδ: _, eГδ: 9, eДε: 2, eЕε: _, eЖε: _, eЗζ: _, eИζ: _, eКζ: 1; 
Missing the numbers - 3, 4, 6, 7, 8. 
The empty squares in gro

 3) We need to fill in numbers 2, 3, 4, 9 in squares dКζ, eИζ, fИζ, fКζ. The intersecting groups are d, e, f, И, К, ζ.

Intersecting group d: dБδ: 2, dВδ: _, dГδ: _, dДε: _, dЕε: 6, dЖε: _, dЗζ: 8, dИζ: 5, dКζ: _;
Intersecting group d contains 2, 5, 6, 8.
It intersects square dКζ, therefore square dКζ cannot be 2, 5, 6, 8.

Intersecting group e: eБδ: 5, eВδ: _, eГδ: 9, eДε: 2, eЕε: 4, eЖε: _, eЗζ: 7, eИζ: _, eКζ: 1;
Intersecting group e contains 1, 2, 4, 5, 7, 9.
It intersects square eИζ, therefore square eИζ cannot be 1, 2, 4, 5, 7, 9.

Intersecting group f: fБδ: 2, fВδ: _, fГδ: _, fДε: 7, fЕε: _, fЖε: 3, fЗζ: 6, fИζ: _, fКζ: _;
Intersecting group f contains 2, 3, 6, 7.
It intersects squares fИζ, fКζ, therefore squares fИζ, fКζ cannot be 2, 3, 6, 7.

Intersecting group И: aИγ: _, bИγ: _, cИγ: 2, dИζ: 5, eИζ: _, fИζ: _, gИι: 6, hИι: 7, iИι: _;
Intersecting group И contains 2, 5, 6, 7.
It intersects squares eИζ, fИζ, therefore squares eИζ, fИζ cannot be 2, 5, 6, 7.

Intersecting group К

 Let's start with step 1:

1) Select 6 groups from all groups in the sudoku. 

After examining the sudoku, I have selected the following groups:

Group В: aВα, bВα, cВα, dВδ, eВδ, fВδ, gВη, hВη, iВη
Group Д: aДβ, bДβ, cДβ, dДε, eДε, fДε, gДθ, hДθ, iДθ
Group З: aЗγ, bЗγ, cЗγ, dЗζ, eЗζ, fЗζ, gЗι, hЗι, iЗι
Group f: fБδ, fВδ, fГδ, fДε, fЕε, fЖε, fЗζ, fИζ, fКζ
Group i: iБη, iВη, iГη, iДθ, iЕθ, iЖθ, iЗι, iИι, iКι
Group θ: gДθ, gЕθ, gЖθ, hДθ, hЕθ, hЖθ, iДθ, iЕθ, iЖθ

1.1) For each of the selected groups above:

Group В -  numbers aВα: 2, bВα: 8, cВα: _, dВδ: _, eВδ: _, fВδ: _, gВη: 7, hВη: 9, iВη: 3;
Missing the numbers - 1, 4, 5, 6.
The empty squares in group В are cВα, dВδ, eВδ, fВδ.
The filled in numbers are 5, the empty squares are 4.

Group Д - contains the numbers aДβ: _, bДβ: 9, cДβ: 8, dДε: _, eДε: 2, fДε: 7, gДθ: _, hДθ: _, iДθ: _; 
Missing the numbers - 1, 3, 4, 5, 6.
The empty squares in group Д are aДβ, dДε, gДθ, hДθ, iДθ.
The filled in numbers are 4, the empty squares are 5.

Gro

 3) We need to fill in numbers 1, 2, 5, 9 in squares bЗγ, cЗγ, hЗι, iЗι. The intersecting groups are b, c, h, i, γ, ι.

Intersecting group b: aБα: _, bВα: 8, bГα: 7, bДβ: 9, bЕβ: 2, bЖβ: _, bЗγ: _, bИγ: _, bКγ: _;
Intersecting group b contains 2, 7, 8, 9.
It intersects square bЗγ, therefore square bЗγ cannot be 2, 7, 8 or 9.

Intersecting group c: cБα: _, cВα: 5, cГα: 4, cДβ: 8, cЕβ: 6, cЖβ: _, cЗγ: _, cИγ: 2, cКγ: 7;
Intersecting group c contains 2, 4, 5, 6, 7, 8.
It intersects square cЗγ, therefore square cЗγ cannot be 2, 4, 5, 6, 7 or 8.

Intersecting group h: hБη: 8, hВη: 9, hГη: 2, hДθ: _, hЕθ: 3, hЖθ: _, hЗι: _, hИι: 7, hКι: _;
Intersecting group h contains 2, 3, 7, 8, 9.
It intersects square hЗι, therefore square hЗι cannot be 2, 3, 7, 8 or 9.

Intersecting group i: iБη: _, iВη: 3, iГη: _, iДθ: _, iЕθ: _, iЖθ: 7, iЗι: _, iИι: _, iКι: 8;
Intersecting group i contains 3, 7, 8.
It intersects square iЗι, therefore square iЗι cannot be 3, 7 or 8.

Intersecting group γ: aБα: _, aВα: 2

 1) Let's select 6 groups from all groups in the sudoku. We will alternate between row, column, 3x3 square and choose the groups that contain a low, but non-zero number of empty squares. We cannot choose any groups mentioned in the recently tested groups list.

The selected groups are:

Group i: iБη, iВη, iГη, iДθ, iЕθ, iЖθ, iЗι, iИι, iКι
Group Б: aБα, bБα, cБα, dБδ, eБδ, fБδ, gБη, hБη, iБη
Group θ: gДθ, gЕθ, gЖθ, hДθ, hЕθ, hЖθ, iДθ, iЕθ, iЖθ
Group f: fБδ, fВδ, fГδ, fДε, fЕε, fЖε, fЗζ, fИζ, fКζ
Group К: aКγ, bКγ, cКγ, dКζ, eКζ, fКζ, gКι, hКι, iКι
Group ι: gЗι, gИι, gКι, hЗι, hИι, hКι, iЗι, iИι, iКι

1.1) For each of the selected groups above:

Group i -  numbers iБη: _, iВη: 3, iГη: _, iДθ: _, iЕθ: _, iЖθ: 7, iЗι: _, iИι: _, iКι: 8;
Missing the numbers - 1, 2, 4, 5, 6, 9.
The empty squares in group i are iБη, iГη, iДθ, iЕθ, iЗι, iИι.
The filled in numbers are 3, the empty squares are 6.

Group Б - contains the numbers aБα: _, bБα: _, cБα: _, dБδ: 7, eБδ: 5, fБδ: 2, gБη: _, hБη: 8, iБη:

 3) Now we have the missing numbers from our target group, the empty squares and the intersecting groups.

We need to fill in numbers 3, 6, 9 in squares aБα, bБα, cБα. The intersecting groups are a, b, c, Б.

Intersecting group a: aБα: _, aВα: 2, aГα: 1, aДβ: _, aЕβ: _, aЖβ: _, aЗγ: 4, aИγ: _, aКγ: 6;
Intersecting group a contains 1, 2, 4, 6.
It intersects square aБα, therefore square aБα cannot be 1, 2, 4, 6.

Intersecting group b: bБα: _, bВα: 8, bГα: 7, bДβ: 9, bЕβ: 2, bЖβ: _, bЗγ: _, bИγ: _, bКγ: _;
Intersecting group b contains 2, 7, 8, 9.
It intersects square bБα, therefore square bБα cannot be 2, 7, 8, 9.

Intersecting group c: cБα: _, cВα: 5, cГα: 4, cДβ: 8, cЕβ: 6, cЖβ: 1, cЗγ: 9, cИγ: 2, cКγ: 7;
Intersecting group c contains 1, 2, 4, 5, 6, 7, 8, 9.
It intersects square cБα, therefore square cБα cannot be 1, 2, 4, 5, 6, 7, 8, 9.

Intersecting group Б: aБα: _, bБα: _, cБα: _, dБδ: 7, eБδ: 5, fБδ: 2, gБη: _, hБη: 8, iБη: _;
Intersecting group Б contains 2, 5, 7, 8.
It intersects

 1) Let's select 6 groups from all groups in the sudoku. We will alternate between row, column, 3x3 square and choose the groups that contain a low, but non-zero number of empty squares. We cannot choose any groups mentioned in the recently tested groups list.

The groups we will select are: 

- Group d: dБδ, dВδ, dГδ, dДε, dЕε, dЖε, dЗζ, dИζ, dКζ
- Group З: aЗγ, bЗγ, cЗγ, dЗζ, eЗζ, fЗζ, gЗι, hЗι, iЗι
- Group θ: gДθ, gЕθ, gЖθ, hДθ, hЕθ, hЖθ, iДθ, iЕθ, iЖθ
- Group i: iБη, iВη, iГη, iДθ, iЕθ, iЖθ, iЗι, iИι, iКι
- Group И: aИγ, bИγ, cИγ, dИζ, eИζ, fИζ, gИι, hИι, iИι
- Group ι: gЗι, gИι, gКι, hЗι, hИι, hКι, iЗι, iИι, iКι

1.1) For each of the selected groups above, we will name the numbers they contain, the numbers remaining, which cells are empty, and how many cells are filled in and how many are empty.

- Group d: numbers dБδ: 7, dВδ: _, dГδ: _, dДε: _, dЕε: _, dЖε: 9, dЗζ: 8, dИζ: 5, dКζ: _; Missing the numbers - 1, 2, 3, 4, 6. The empty squares in group d are dВδ, dГδ, dДε, dЕε, dКζ. T

 3) We need to fill in the number 8 in square eЖε. The intersecting groups are Ж, ε.

Intersecting group Ж: aЖβ: _, bЖβ: _, cЖβ: 1, dЖε: 9, eЖε: _, fЖε: 3, gЖθ: _, hЖθ: _, iЖθ: 7; 
Intersecting group Ж contains 1, 3, 7, 9. It intersects square eЖε, therefore square eЖε cannot be 1, 3, 7, 9.

Intersecting group ε: dДε: _, dЕε: _, dЖε: 9, eДε: 2, eЕε: 4, eЖε: _, fДε: 7, fЕε: _, fЖε: 3; 
Intersecting group ε contains 2, 3, 4, 7, 9. It intersects square eЖε, therefore square eЖε cannot be 2, 3, 4, 7, 9.

3.1) Referring to step 3, square eЖε cannot be 1, 2, 3, 4, 7, 9.

4) Square eЖε: initial possibilities [8]. From them, for square eЖε, we have excluded the possibilities [1, 2, 3, 4, 7, 9]. Therefore, the final possibilities for eЖε are [8].

We can fill in number 8 in square eЖε.

4.1) There is only one possibility for square eЖε, we can fill it in! Square eЖε = 8.

5) Recently tested groups (old): З, Б, α, f, c, З, В, Г, ζ, e.
Recently tested groups: e, З, Б, α, f, c, З, В, Г, ζ.

6) We 

In [ ]:
# restart the process from the last valid grid

# grids = grids[:-1]
# take_steps(maximum_steps = 50, current_step = 9)

In [ ]:
# for debug purposes 

print(generate_user_prompt(grids[-2]))

In [ ]:
# for debug purposes

print(len(grids))
print(grids[-2])

process_str_grid(grids[-1])
# processed_grid = process_str_grid(new_grid)
# correct, problem = is_valid_sudoku(processed_grid)

In [ ]:
# for debug purposes

print(len(answers))
print(answers[-1])